In [19]:
import models
import pandas as pd

In [20]:
# METRIC = 'F1'
# ALGO = 'detection_correction'
# # ALGO = 'correction'
MODEL = 'RNN_128_units'
COMMODITY = 'copper_over'
confidence_levels = [0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

algo = ['correction', 'detection_correction']

In [21]:
# df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'copper/test/predictions/test_{METRIC}_{ALGO}/', confidence_levels)
df = models.npy_to_based_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'{COMMODITY}/test/predictions/test_F1_correction', confidence_levels)
df

c:\Users\nocet\OneDrive\Documents\Projects\lab-v2\ezra\analysis\models.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  bowpy_dataframe[f"rule{index}"] = model_predictions['Predicted']


,pred,corr,true_positive,false_positive,rule0,rule1,rule2,rule3,rule4,rule5,...,rule422,rule423,rule424,rule425,rule426,rule427,rule428,rule429,rule430,rule431
0,0,0,0,0,1,1,1,1,1,1,...,0,1,1,0,0,0,0,0,0,0
1,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,0,0,0,0,1,1,1,1,1,1,...,0,0,1,1,0,1,0,1,0,1
218,0,0,0,0,1,1,1,1,1,1,...,1,0,1,1,0,1,0,1,0,1
219,0,0,0,0,1,1,1,1,1,1,...,1,0,1,1,0,1,0,1,0,1
220,0,0,0,0,1,1,1,1,1,1,...,0,0,1,1,0,1,0,1,0,1


In [22]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [23]:
f1, recall, precision = calculate_metrics(df, "pred")
print(f'Base model results\nf1: {f1}\nrecall: {recall}\nprecision: {precision}')

Base model results
f1: 0.2641509433962264
recall: 0.2
precision: 0.3888888888888889


In [24]:
df.to_numpy().dump('edcr/data/test.npy')